In [ ]:
import torch
import os
from itertools import product
from random import shuffle
from tqdm import tqdm

from KNN_Embeddings import *

In [ ]:
# Feature normalization
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset

# Create a scaler object
scaler = StandardScaler()

# Fit on training data and transform both training and test data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Creating DataLoader instances
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## CNNH Model 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNH(nn.Module):
    def __init__(self, input_dim, num_bits):
        super(CNNH, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.relu1 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_bits)  # Outputs the hash codes directly

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNH(input_dim=X_train.shape[1], num_bits=48).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


## Train CNNH Model

In [ ]:
# Training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    for epoch in range(10):
        total_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            hash_codes = torch.tanh(outputs)  # Ensuring outputs are between -1 and 1
            similarity_matrix = torch.mm(hash_codes, hash_codes.t())
            target_similarity = (target[:, None] == target[None, :]).float() * 2 - 1
            loss = criterion(similarity_matrix, target_similarity)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

# Training the model
train(model, train_loader, optimizer, criterion, device)



## KNN on hashed embeddings

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize


def get_binary_hash_codes(model, loader, device):
    model.eval()
    hash_codes = []
    labels = []
    with torch.no_grad():
        for data, target in loader:
            data = data.to(device)
            outputs = model(data)
            binary_codes = torch.sign(outputs).cpu().numpy()  # Convert to binary hash codes
            hash_codes.extend(binary_codes)
            labels.extend(target.numpy())
    return np.array(hash_codes), np.array(labels)

# Extract hash codes
train_codes, train_labels = get_binary_hash_codes(model, train_loader, device)
test_codes, test_labels = get_binary_hash_codes(model, test_loader, device)

# Classification with KNN
knn = KNeighborsClassifier(n_neighbors=5, metric='hamming')
knn.fit(train_codes, train_labels)
predictions = knn.predict(test_codes)
y_pred_proba = knn.predict_proba(test_codes)

print(classification_report(test_labels, predictions))


# Binarize the labels for a one-vs-rest computation
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))

# Calculate the average precision for each class
average_precisions = []
for i in range(y_test_binarized.shape[1]):  # iterate over classes
    average_precisions.append(average_precision_score(y_test_binarized[:, i], y_pred_proba[:, i]))

# Compute the mean of the average precisions
map_score = np.mean(average_precisions)
print(f'Mean Average Precision (MAP): {map_score}')

print(confusion_matrix(test_labels, predictions))




In [ ]:

# Confusion matrix data for another visualization
from matplotlib import pyplot as plt


another_confusion_matrix = np.array([
    [991, 0, 2, 1, 0, 0, 0, 0, 5, 1],
    [1, 984, 0, 0, 0, 0, 0, 0, 0, 15],
    [4, 0, 984, 2, 1, 3, 4, 2, 0, 0],
    [1, 2, 5, 968, 3, 14, 4, 0, 0, 3],
    [0, 0, 6, 7, 984, 0, 2, 0, 0, 1],
    [0, 3, 1, 25, 4, 967, 0, 0, 0, 0],
    [3, 0, 1, 4, 1, 0, 991, 0, 0, 0],
    [0, 0, 0, 3, 7, 0, 0, 990, 0, 0],
    [5, 0, 0, 0, 0, 2, 0, 0, 993, 0],
    [0, 6, 1, 0, 0, 0, 0, 0, 2, 991]
])

fig, ax = plt.subplots(figsize=(10, 8))
cax = ax.matshow(another_confusion_matrix, cmap=plt.cm.Blues)  # Using Cividis colormap

# Adding the color bar
plt.colorbar(cax)

# Setting axis labels for a 10-class problem
classes = list(range(10))  # assuming the classes are labeled from 0 to 9
ax.set_xticklabels([''] + classes)
ax.set_yticklabels([''] + classes)

# Adding text annotation
for (i, j), val in np.ndenumerate(another_confusion_matrix):
    ax.text(j, i, f'{val}', ha='center', va='center', color='white' if val > 500 else 'black')

ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)
ax.set_title('Confusion Matrix', fontsize=14, fontweight='bold')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib

def visualize_embeddings(embeddings, labels, num_classes):
    # Convert labels from tensor to numpy if not already
    labels = labels.numpy() if not isinstance(labels, np.ndarray) else labels

    # Initialize t-SNE
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(embeddings)

    # Create a figure with high resolution
    plt.figure(figsize=(10, 7), dpi=300)  # Set DPI for high resolution
    cmap = plt.cm.get_cmap('tab20b', num_classes)  # Use a more distinct colormap

    # Plotting the results of t-SNE
    scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=labels, cmap=cmap, marker='o', edgecolor='k', alpha=0.6)
    plt.colorbar(scatter, ticks=range(num_classes))
    plt.title('t-SNE visualization of Embeddings', fontsize=14, fontweight='bold')
    plt.xlabel('t-SNE axis 1', fontsize=12)
    plt.ylabel('t-SNE axis 2', fontsize=12)
    plt.grid(True, linestyle='--', linewidth=0.5)  # Lighter grid
    plt.show()

# Usage example
train_embeddings, train_labels = get_binary_hash_codes(model, test_loader, device)  # Make sure these are on CPU
visualize_embeddings(train_embeddings, train_labels, len(np.unique(train_labels)))


In [ ]:
import matplotlib.pyplot as plt

# Data for CIFAR-10 and PathMNIST
methods = ['DSH', 'CNNH', 'DPSH', 'DHN', 'HashNet', 'DSDH']
cifar10_results = {
    'DSH': [88.78, 88.93, 88.99, 91.03],
    'CNNH': [97.88, 98.05, 98.08, 98.14],
    'DPSH': [88.74, 91.43, 92.28, 95.56],
    'DHN': [87.13, 93.07, 93.96, 95.19],
    'HashNet': [97.74, 98.81, 97.90, 97.98],  # Correction assumed for 36bits (07.90 -> 97.90)
    'DSDH': [97.73, 98.05, 98.04, 98.03]
}

pathmnist_results = {
    'DSH': [51.54, 52.89, 51.62, 57.20],
    'CNNH': [23.40, 26.64, 26.72, 27.89],
    'DPSH': [66.74, 67.91, 69.02, 69.56],
    'DHN': [69.09, 69.64, 69.98, 69.80],
    'HashNet': [31.31, 33.24, 47.44, 49.79],
    'DSDH': [68.19, 69.33, 69.81, 70.15]
}

bits = [12, 24, 36, 48]

# Plotting for CIFAR-10
plt.figure(figsize=(14, 7))
for method, results in cifar10_results.items():
    plt.plot(bits, results, marker='o', label=method)

plt.title('MAP Trends on CIFAR-10 for Different Bit Lengths')
plt.xlabel('Bit Length')
plt.ylabel('Mean Average Precision (MAP)')
plt.legend()
plt.grid(True)
plt.show()

# Plotting for PathMNIST
plt.figure(figsize=(14, 7))
for method, results in pathmnist_results.items():
    plt.plot(bits, results, marker='o', label=method)

plt.title('MAP Trends on PathMNIST for Different Bit Lengths')
plt.xlabel('Bit Length')
plt.ylabel('Mean Average Precision (MAP)')
plt.legend()
plt.grid(True)
plt.show()
